In [5]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from tensorflow import keras
from tqdm import tqdm

In [6]:
import tensorflow.keras.backend as K

## Считаем препроцессные данные из прошлого ноутбучка

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
interactions_df = pd.read_csv('/content/drive/MyDrive/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/drive/MyDrive/users_processed_kion.csv')
items_df = pd.read_csv('/content/drive/MyDrive/items_processed_kion.csv')

In [9]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,M,True
1,962099,age_18_24,income_20_40,M,False
2,1047345,age_45_54,income_40_60,F,False
3,721985,age_45_54,income_20_40,F,False
4,704055,age_35_44,income_60_90,F,False


In [10]:
items_df = items_df.rename(columns = {'id' : 'item_id'})

In [11]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [12]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


## Готовим фичи пользователей

Посмотрим, какие фичи в датасете фильмов являются категориальными и закодируем их с помощью one-hot encoding.

In [13]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом,
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


## Готовим фичи айтемов

Кодируем их точно так же - one-hot'ом.

In [14]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [15]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating',
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
0,10711,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Сделаем матрицу взаимодействий

In [16]:
interactions_df.item_id.value_counts()

10440    202457
15297    193123
9728     132865
13865    122119
4151      91167
          ...  
8076          1
8954          1
15664         1
818           1
10542         1
Name: item_id, Length: 15706, dtype: int64

In [17]:
interactions_df.user_id.value_counts()

416206     1341
1010539     764
555233      685
11526       676
409259      625
           ... 
45493         1
615194        1
96848         1
425823        1
697262        1
Name: user_id, Length: 962179, dtype: int64

In [18]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [19]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6901


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [20]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72,10616,3944
1,699317,1659,2021-05-29,8317,100,42131,675
6,1016458,354,2021-08-14,1672,25,61024,139
7,884009,693,2021-08-04,703,14,53150,279
14,5324,8437,2021-04-18,6598,92,310,3485


# 1.1 Задание Использовать информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования

In [26]:
# Convert last_watch_dt to datetime format
interactions_df['last_watch_dt'] = pd.to_datetime(interactions_df['last_watch_dt'])

# Now we can use the .dt accessor
interactions_vec = np.zeros((interactions_df.uid.nunique(), interactions_df.iid.nunique()))

# Iterate over interactions and use last_watch_dt for more representative sampling
for i, (user_id, item_id, last_watch_dt) in enumerate(zip(interactions_df.uid, interactions_df.iid, interactions_df.last_watch_dt)):
    interactions_vec[user_id, item_id] += last_watch_dt.timestamp()

# Normalize the rows of the interactions_vec matrix
res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]


In [27]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6901
65974
65974
{11805, 9788, 11501, 1734}


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [28]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [29]:
# Mapping item IDs to IIDs
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid.get(x, "Unknown"))

# Setting "Unknown" items as a default value or handle it according to your logic
items_ohe_df = items_ohe_df.set_index("iid")

# Mapping user IDs to UIDs
users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid.get(x, "Unknown"))

# Setting "Unknown" users as a default value or handle it according to your logic
users_ohe_df = users_ohe_df.set_index("uid")


In [30]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный

    return loss

## Генератор и семплирование



In [31]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0] - 1)

            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])

            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))

            # фичи юзера
            uid_meta.append(users.iloc[uid_i])

            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])

            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])

            # фичи плохого айтема
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [
            np.array(uid_meta),
            np.array(uid_interaction),
        ]

In [32]:
# Initialize the generator
gen = generator(items=items_ohe_df.drop(["item_id"], axis=1),
                users=users_ohe_df.drop(["user_id"], axis=1),
                interactions=interactions_vec)

# Get a batch from the generator
batch = next(gen)
# Access the components of the batch
uid_meta, uid_interaction, pos, neg = batch[0]

print(f"Vector of user features: {uid_meta.shape}")
print(f"Vector of user interactions with items: {uid_interaction.shape}")
print(f"Vector of 'good' item features: {pos.shape}")
print(f"Vector of 'bad' item features: {neg.shape}")
print('-----------------------------------------------------')


Vector of user features: (1024, 19)
Vector of user interactions with items: (1024, 6897)
Vector of 'good' item features: (1024, 8588)
Vector of 'bad' item features: (1024, 8588)
-----------------------------------------------------


## Model Architecture Setup

In [33]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], )
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (8588,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [34]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])

    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)


    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)

# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [35]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")


In [36]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [37]:
# модель айтема
item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 8588)]               0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 128)                  1099264   ['input_8[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 128)                  16384     ['dense_7[0][0]']             
                                                                                                  
 add_2 (Add)                 (None, 128)                  0         ['dense_7[0][0]',             
                                                                     'dense_8[0][0]']       

In [38]:
# модель юзера
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 128)                  2432      ['input_9[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 128)                  16384     ['dense_10[0][0]']            
                                                                                                  
 input_10 (InputLayer)       [(None, 6897)]               0         []                            
                                                                                            

In [39]:
# общая модель
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 6897)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 8588)]               0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 8588)]               0         []                            
                                                                                            

# Start the training

In [ ]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1),
                    users=users_ohe_df.drop(["user_id"], axis=1),
                    interactions=interactions_vec,
                    batch_size=64),
          steps_per_epoch=100,
          epochs=30,
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/30
100/100 [==============================] - 31s 293ms/step - loss: 0.3037 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 24s 235ms/step - loss: 0.2180 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 31s 312ms/step - loss: 0.2005 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 25s 250ms/step - loss: 0.1871 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 35s 348ms/step - loss: 0.1791 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 21s 208ms/step - loss: 0.1768 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 21s 207ms/step - loss: 0.1707 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 21s 213ms/step - loss: 0.1688 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 22s 217ms/step - loss: 0.1540 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 23s 231ms/step - loss: 0.1622 - lr: 0.0010
Epoch 11/30
100/100 [========

In [ ]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[int(rand_uid)]
user_interaction_vec = interactions_vec[int(rand_uid)]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[int(rand_iid)]

# получаем вектор юзера
user_vec = u2v.predict(
    [np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)]
)

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))
from sklearn.metrics.pairwise import euclidean_distances as ED
# считаем расстояние между вектором юзера и вектором айтема
ED(user_vec, item_vec)


1/1 [==============================] - 0s 110ms/step


array([[1.6187288]], dtype=float32)

In [ ]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

216/216 [==============================] - 1s 6ms/step


In [ ]:
def recommendations(user_id,items_vecs, top_n=10):
    # Calculate Euclidean distances
    dists = ED(user_vec, items_vecs)

    # Get indices of top N items with smallest distances
    top_indices = np.argsort(dists)[0, :top_n]

    # Get corresponding item IDs
    recommended_item_ids = items_ohe_df.iloc[top_indices]['item_id'].tolist()

    return recommended_item_ids


# Getting recomendations

In [ ]:
recos = {}
users = list(user_id_to_uid.keys())
for user_id in tqdm(users):

    recommended_items = recommendations(user_id, items_vecs )
    print(f"Recommended items for user {user_id}: {recommended_items}")


 69%|██████▉   | 45686/65974 [15:34<07:16, 46.46it/s]


Recommended items for user 610626: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 170635: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 1074771: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 121459: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 667903: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 813460: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 58908: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 667726: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 696498: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]


 69%|██████▉   | 45696/65974 [15:35<07:07, 47.43it/s]

Recommended items for user 623565: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 351999: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 421445: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 1068936: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 634748: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 453653: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 572895: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 475318: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 288572: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 985029: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]


 69%|██████▉   | 45706/65974 [15:35<07:02, 47.95it/s]

...


Recommended items for user 151005: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 361820: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 497786: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 701748: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 582849: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 991140: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 309360: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 134709: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 1035117: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]


100%|█████████▉| 65964/65974 [23:00<00:00, 45.58it/s]

Recommended items for user 602904: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 673680: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 627907: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 806810: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 58141: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 912817: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 626830: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 807545: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 1001692: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 628833: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]


100%|██████████| 65974/65974 [23:00<00:00, 47.79it/s]

Recommended items for user 417555: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 945345: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 1097192: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 126736: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 424247: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 736266: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 679795: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]
Recommended items for user 30545: [8532, 15771, 3929, 826, 7314, 4702, 14017, 13582, 3585, 10281]


In [ ]:
import numpy as np
from tqdm import tqdm

def recommend_items_to_user(user_id, item_embeddings, topn=10, sample_size=100):
    # Map user_id to internal user index
    internal_user_index = user_id_to_uid[user_id]

    # Introduce randomness in user selection
    random_user_index = np.random.choice(list(users_ohe_df.index))

    # Extract user metadata features and interaction vector
    user_metadata_features = users_ohe_df.drop(["user_id"], axis=1).iloc[random_user_index]
    user_interaction_vector = interactions_vec[random_user_index]

    # Predict user vector using the trained user-to-vector model
    user_vector = u2v.predict(
        [np.array(user_metadata_features).reshape(1, -1), np.array(user_interaction_vector).reshape(1, -1)],
        verbose=False,
    )

    # Instead of calculating distance for all items, just select a random subset
    sampled_item_indices = np.random.choice(item_embeddings.shape[0], size=sample_size, replace=False)
    sampled_item_embeddings = item_embeddings[sampled_item_indices, :]

    # Calculate distances between the user vector and sampled item embeddings
    distances = ED(user_vector, sampled_item_embeddings)

    # Get the indices of the topn items from the sampled set
    topn_item_indices_sampled = np.argsort(distances, axis=1)[:, :topn]

    # Map internal item indices to item_ids
    topn_item_ids = [iid_to_item_id[iid] for iid in topn_item_indices_sampled.reshape(-1)]

    return topn_item_ids



In [ ]:
# Dictionary to store recommendations for each user
recommendations_dict = {}

# Iterate over all users to generate recommendations
for user_id in tqdm(users):
    recommendations_for_user = recommend_items_to_user(user_id, items_vecs)
    recommendations_dict.update({user_id: recommendations_for_user})


  0%|          | 38/65974 [00:05<2:41:17,  6.81it/s]


KeyboardInterrupt: ignored

In [ ]:
print(recommendations_dict)

 Runtime restarted many times when trying to get recomendation